In [ ]:
NBR_SERVERS = 1
LAMBDA = 1 #intensité processus d'arrivées : nombre moyen d'arrivées par unité de temps
MU = 1.3 / NBR_SERVERS #paramètre du temps de service (exponentielle) : nombre moyen de clients pouvant être traités par unité de temps
NBR_RUNS = 20
CONFIDENCE_LEVEL = 95 #pourcentage donnant le niveau de confiance des résultats
TOTAL_DURATION = 20000 #durée totale simulée
TRANSIENT_DURATION = 10000 #temps estimé pour arriver en régime transient

In [ ]:
import random

def interarrival_time():
    return random.expovariate(LAMBDA)

def service_time():
    return random.expovariate(MU)

In [ ]:
"Simulation d'une M/M/m"

import heapq #on utilise une heap pour maintenir l'échéancier ; y place un élément en O(log n), récupère le premier en O(1)

#classe Système, contient le système, mais aussi l'échéancier

class System(object):
     
    def __init__(self):
        self.schedule = [] #échéancier
        self.queue = Queue() #file d'attente
        self.clients = [] #la liste des clients
        
#classe client
        
class Client(object):
    
    def __init__(self, i):
        self.id = i #identifiant du client
        self.arrival = -1 #vaut -1 à l'arrivée
        self.end_waiting = -1
        self.departure = -1 #vaut -1 tant qu'il n'est pas reparti

#classe file d'attente
        
class Queue(object):
    
    def __init__(self):
        self.attente = [] #contient la liste des clients en attente
        self.service = [] #contient les clients en service
        
#classe événement

class Event(object):
    
    def __init__(self, time):
        self.time = time
        self.type = ""

#sous-classe "fin de simulation"

class Event_end(Event):
    
    def __init__(self, time):
        self.time = time
        self.type = "fin"

#sous-classe "arrivée d'un client"

class Event_arrival(Event):
    
    def __init__(self, time):
        self.time = time
        self.type = "arrival"
    
    def action(self, sys):
        client = Client(len(sys.clients))
        client.arrival = self.time
        sys.clients.append(client)
        
        if (len(sys.queue.service) == NBR_SERVERS):
            sys.queue.attente.append(client)
        else:
            client.end_waiting = self.time
            sys.queue.service.append(client)
            e_service = Event_end_service(client.id, self.time + service_time())
            heapq.heappush(sys.schedule, (e_service.time, e_service))
            
            
        e_next_arrival = Event_arrival(self.time + interarrival_time())
        heapq.heappush(sys.schedule, (e_next_arrival.time, e_next_arrival)) 


                
#sous-classe "départ d'un client"
            
class Event_end_service(Event):
    
    def __init__(self, i, time):
        self.time = time
        self.type = "service"
        self.client_id = i
    
    def action(self, sys):
        i = 0;
        while (sys.queue.service[i].id != self.client_id):
            i = i + 1
        client = sys.queue.service.pop(i)
               
        client.departure = self.time  
        
        if len(sys.queue.attente):
            next_client = sys.queue.attente.pop(0)
            next_client.end_waiting = self.time
            sys.queue.service.append(next_client)
            e_service = Event_end_service(next_client.id, self.time + service_time())
            heapq.heappush(sys.schedule, (e_service.time, e_service))            

In [ ]:
"Lance la simulation, nombre de réplications donné par NBR_RUNS"

from statistics import mean 

waiting_times_run = [] #contient le temps moyen d'attente pour chaque run
sojourn_times_run = [] #contient le temps moyen de séjour pour chaque run

for i in range(NBR_RUNS):
    random.seed(i) #permet de la reproductibilité
    sys = System()
    e_fin = Event_end(TOTAL_DURATION)
    heapq.heappush(sys.schedule, (e_fin.time, e_fin))
    e_debut = Event_arrival(0)
    heapq.heappush(sys.schedule, (e_debut.time, e_debut))
    
    while sys.schedule[0][1].type != "fin":
        (time, e) = heapq.heappop(sys.schedule)
        e.action(sys)
    
    waiting_times_clients = [] #temps d'attente par client pour le run i courant
    sojourn_times_clients = [] #temps de séjour par client pour le run i courant
    
    for j in range(len(sys.clients)):
        #on ne garde que les clients qui sont arrivés en régime permanent et qui ont quitté la file
        if sys.clients[j].departure != -1 and sys.clients[j].arrival >= TRANSIENT_DURATION:
            waiting_times_clients.append(sys.clients[j].end_waiting - sys.clients[j].arrival)
            sojourn_times_clients.append(sys.clients[j].departure - sys.clients[j].arrival)
    
    waiting_times_run.append(mean(waiting_times_clients))
    sojourn_times_run.append(mean(sojourn_times_clients))

In [ ]:
"Fait l'analyse"

from scipy.stats import t, sem

waiting_mean = mean(waiting_times_run)
waiting_sample_standard_error = sem(waiting_times_run)
confidence_interval = t.interval(CONFIDENCE_LEVEL / 100, NBR_RUNS - 1, waiting_mean, waiting_sample_standard_error)

print("Temps moyen d'attente : %.4f" % waiting_mean)
print("Intervalle de confiance à %d%%: " % CONFIDENCE_LEVEL, end="") ; print("(%.4f, %.4f)" % confidence_interval) 

sojourn_mean = mean(sojourn_times_run)
sojourn_sample_standard_error = sem(sojourn_times_run)
confidence_interval = t.interval(CONFIDENCE_LEVEL / 100, NBR_RUNS - 1, sojourn_mean, sojourn_sample_standard_error)

print("Temps moyen de séjour : %.4f" % sojourn_mean)
print("Intervalle de confiance à %d%%: " % CONFIDENCE_LEVEL, end="") ; print("(%.4f, %.4f)" % confidence_interval) 

In [ ]:
import matplotlib.pyplot as plt

average_sojourn_times = []
total = 0

sojourn_times_clients = []

for j in range(len(sys.clients)):
    if sys.clients[j].departure != -1:
        sojourn_times_clients.append(sys.clients[j].departure - sys.clients[j].arrival)

plt.plot(sojourn_times_clients)
plt.show()

for j in range(len(sojourn_times_clients)):
    total += sojourn_times_clients[j]
    average_sojourn_times.append(total / (j+1))

print("Moyenne empirique : %.4f" % average_sojourn_times[len(sojourn_times_clients)-1])

plt.plot(average_sojourn_times)
plt.show()